#4

In [1]:
class Animal:
    def __init__(self, name, old) -> None:
        self.name = name
        self.old = old

class Cat(Animal):
    def __init__(self, name, old, color, weight) -> None:
        super().__init__(name, old)
        self.color = color
        self.weight = weight

    def get_info(self):
        return f'{self.name}: {self.old}, {self.color}, {self.weight}'

class Dog(Animal):
    def __init__(self, name, old, breed, size) -> None:
        super().__init__(name, old)
        self.breed = breed
        self.size = size

    def get_info(self):
        return f'{self.name}: {self.old}, {self.breed}, {self.size}'


#5

In [2]:
class Thing:
    ID=1

    def __init__(self, name, price) -> None:
        self.name = name
        self.price = price
        self.id = Thing.ID
        Thing.ID += 1
        self.weight = None
        self.dims = None
        self.memory = None
        self.frm = None

class Table(Thing):
    def __init__(self, name, price, weight, dims) -> None:
        super().__init__(name, price)
        self.weight = weight
        self.dims = dims

class ElBook(Thing):
    def __init__(self, name, price, memory, frm) -> None:
        super().__init__(name, price)
        self.memory = memory
        self.frm = frm

#6

In [3]:
class GenericView:
    def __init__(self, methods=('GET')):
        self.methods = methods

    def get(self, request):
        return ""

    def post(self, request):
        pass

    def put(self, request):
        pass

    def delete(self, request):
        pass

class DetailView(GenericView):
    def get(self, request):
        if type(request) is not dict:
            raise TypeError('request не является словарем')

        if 'url' not in request:
            raise TypeError('request не содержит обязательного ключа url')

        return f"url: {request['url']}"


    def render_request(self, request, method):
        if method.upper() not in self.methods:
            raise TypeError('данный запрос не может быть выполнен')

        return self.__getattribute__(method.lower())(request)

#7

In [4]:
class Singleton:
    __instance = None

    def __new__(cls, *args, **kwargs):
        if cls.__instance:
            return cls.__instance
        
        cls.__instance = super().__new__(cls)
        return cls.__instance

class Game(Singleton):
    def __init__(self, name) -> None:
        if 'name' not in self.__dict__:
            self.name = name        

#8

In [5]:
class Validator:
    def _is_valid(self, data):
        return True

    def __call__(self, data):
        if not self._is_valid(data):
            raise ValueError('данные не прошли валидацию')

class IntegerValidator(Validator):
    def __init__(self, min_value, max_value) -> None:
        self.min_value = min_value
        self.max_value = max_value

    def _is_valid(self, data):
        return True if type(data) is int and self.min_value <= data <= self.max_value else False
        
class FloatValidator(Validator):
    def __init__(self, min_value, max_value) -> None:
        self.min_value = min_value
        self.max_value = max_value
    
    def _is_valid(self, data):
        return True if type(data) is float and self.min_value <= data <= self.max_value else False


In [6]:
integer_validator = IntegerValidator(-10, 10)
float_validator = FloatValidator(-1, 1)
res1 = integer_validator(10)  # исключение не генерируется (проверка проходит)
res2 = float_validator(10)    # исключение ValueError

ValueError: данные не прошли валидацию

#9

In [7]:
class Layer:
    def __init__(self) -> None:
        self.name = 'Layer'
        self.next_layer = None

    def __call__(self, layer):
        self.next_layer = layer
        return layer

class Input(Layer):
    def __init__(self, inputs) -> None:
        super().__init__()
        self.inputs = inputs
        self.name = 'Input'

    def __call__(self, layer):
        self.next_layer = layer
        return layer


class Dense(Layer):
    def __init__(self, inputs, outputs, activation) -> None:
        super().__init__()
        self.inputs = inputs
        self.outputs = outputs
        self.activation = activation
        self.name = 'Dense'

    def __call__(self, layer):
        self.next_layer = layer
        return layer

class NetworkIterator:
    def __init__(self, network) -> None:
        self.network = network

    def __iter__(self):
        return self

    def __next__(self):
        curr = self.network
        if curr:
            self.network = self.network.next_layer
            return curr
        
        raise StopIteration



In [8]:
network = Input(128)
layer = network(Dense(network.inputs, 1024, 'linear'))
layer = layer(Dense(layer.inputs, 10, 'softmax'))


In [9]:
for x in NetworkIterator(network):
    print(x.name)


Input
Dense
Dense


#10

In [10]:
class Vector:
    def __init__(self, *args) -> None:
        if not self.__check(args):
            raise ValueError
        self.coords = args
    
    @staticmethod
    def __check(coords):
        if all([type(x) in (int, float) for x in coords]):
            return True

    def get_coords(self):
        return self.coords

    def __add__(self, another_vector):
        if type(another_vector) is Vector and len(self.coords) == len(another_vector.coords):
            return Vector(*[self.coords[i] + another_vector.coords[i] for i in range(len(self.coords))])

    def __sub__(self, another_vector):
        if type(another_vector) is Vector and len(self.coords) == len(another_vector.coords):
            return Vector(*[self.coords[i] - another_vector.coords[i] for i in range(len(self.coords))])


class VectorInt(Vector):
    def __init__(self, *args) -> None:
        if not self.__check(args):
            raise ValueError('координаты должны быть целыми числами')
        super().__init__(*args)

    @staticmethod
    def __check(coords):
        if all([type(x) is int for x in coords]):
            return True

    def __add__(self, another_vector):
        if self.__check(another_vector.coords) and len(self.coords) == len(another_vector.coords):
            return VectorInt(*[self.coords[i] + another_vector.coords[i] for i in range(len(self.coords))])

        return super().__add__(another_vector)

    def __sub__(self, another_vector):
        if self.__check(another_vector.coords) and len(self.coords) == len(another_vector.coords):
            return VectorInt(*[self.coords[i] - another_vector.coords[i] for i in range(len(self.coords))])

        return super().__sub__(another_vector)
